In [0]:
WITH onlybaseentities as (
  SELECT
    child
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmparentchildhierarchy`
  WHERE
    adjacencypath like 'Entity|Shared_Mbr_Entity_Structure|Management|BU_NA|%'
    and IsBaseRecord = 1
),
branchnames as (
  SELECT
    backendname,
    alias
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmdimensionmetadata`
),
voluntaryturnover as (
  SELECT
    `Calendar Date`,
    branchname as Entity,
    --grouping(branchname) as grp_branch,
    sum(ftevoluntaryturnover) as Value
  FROM
    (
      SELECT
        TRUNC(TO_DATE(CAST(dateint AS STRING), 'yyyyMMdd'), 'MONTH') as `Calendar Date`,
        branch.alias as branchname,
        ftevoluntaryturnover
      FROM
        `NA-CDP-BU-HR-DEV`.curated.voluntaryturnover as vt
          LEFT JOIN branchnames as branch
            on branch.backendname = vt.EPMBranch
      WHERE
        vt.EPMBranch NOT IN ('100020', '100023')
    )
  GROUP BY
    GROUPING SETS (`Calendar Date`,branchname),(`Calendar Date`)
)

SELECT
  *,
  'Actuals' as Scenario,
  'All LOB' as LOB,
  'Voluntary Turnover' as Account,
  'Voluntary Turnover' as Source
FROM
  voluntaryturnover
WHERE
  `Calendar Date` >= make_date(2023, 10, 1)